In [21]:
import json
import os

def read_data(datafile=os.path.join(os.getcwd(), '.data', 'master.json')):
    with open(datafile, mode='r') as f:
        lines = [json.loads(l) for l in f.readlines()]
        return lines

data = read_data()

print(data[:10])

[{'feed_name': 'The Economist - Science and Technology', 'is_blog': False, 'img_loc': 'http://cdn.static-economist.com/sites/default/files/images/print-edition/20180217_STD001_0.jpg', 'pub_datetime': '2018-02-15 15:48:17', 'summary': 'science fiction is littered with examples of intelligent computers, from hal 9000 in 2001: a space odyssey to eddie in the hitchhikers guide to the galaxy. one thing such fictional machines have in common is a tendency to go wrong, to the detriment of the characters in the story. hal murders most of the crew of a mission to jupiter. eddie obsesses about trivia, and thus puts the spacecraft he is in charge of in danger of destruction. in both cases, an attempt to build something useful and helpful has created a monster.  successful science fiction necessarily plays on real hopes and fears. in the 1960s and 1970s, when hal and eddie were dreamed up, attempts to create artificial intelligence (ai) were floundering, so both hope and fear were hypothetical. bu

In [72]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
from collections import Counter
import numpy as np

init_notebook_mode(connected=True)
kwds = []
[kwds.extend(d['kwds']) for d in data]
pltdata = [go.Histogram(x=kwds, histnorm='probability')]
plot(pltdata)

'file:///home/cmustard/Projects/spyndicated/spyndicated/temp-plot.html'

In [75]:
import numpy as np
from gensim import corpora, models
init_notebook_mode(connected=True)
texts = [d['kwds'] for d in data]
sample_ids = [idd for idd in np.random.choice(range(len(texts)), 600, replace=True)]
sampled = [texts[i] for i in sample_ids]
unsampled = [texts[j] for j in range(len(texts)) if j not in sample_ids]

d = corpora.Dictionary(sampled)
unsampled_bows = [d.doc2bow(u) for u in unsampled]
sampled_corpus = [d.doc2bow(s) for s in sampled]
lda = models.ldamodel.LdaModel(corpus=sampled_corpus, num_topics=60)
blog_entries = [d for d in data if d['is_blog'] is True]
news_entries = [d for d in data if d['is_blog'] is not True]
print("Number of Blog Entries: ", len(blog_entries))
print("Number of News Entries: ", len(news_entries))


Number of Blog Entries:  409
Number of News Entries:  995


In [74]:
unsampled_tops = []
for bow in unsampled_bows:
    unsampled_tops.append(lda.get_document_topics(bow))
    
blogtops = []
protops = []
for t in range(len(unsampled_tops)):
    if data[t]['is_blog'] is True:
        blogtops.extend([i[0] for i in unsampled_tops[t]])
    else:
        protops.extend([i[0] for i in unsampled_tops[t]])

layout = go.Layout(barmode='overlay', title='Topic Probabilities by Source Type')
BlogTopics = go.Histogram(x=blogtops, name='Blog Topics', histnorm='probability', opacity=0.75)
NewsTopics = go.Histogram(x=protops, name='News Topics', histnorm='probability', opacity=0.75)
traces = [BlogTopics, NewsTopics]
fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [70]:
topsterms = []
for u in unsampled_tops:
    topsterms.extend([lda.get_topic_terms(ut[0]) for ut in u])
